In [1]:
import numpy as np
import pickle as pkl

def load_data(path):
    with open(path, 'rb') as f:
        data = pkl.load(f)
    return data

# Load the data
Interaction_matrices = load_data('interaction_matrices.pkl')
spectral_data = load_data('spectra_dataset.pkl')



In [5]:
print(Interaction_matrices.shape)
print(spectral_data.shape)

torch.Size([10000, 6, 6])
torch.Size([10000, 20000])


In [12]:
# reshaping to flatten both piece of data

Interaction_matrices_flattened =Interaction_matrices.reshape(Interaction_matrices.shape[0], -1)
print(Interaction_matrices_flattened.shape)
print(Interaction_matrices_flattened[0])

spectral_data_flattened = spectral_data.reshape(10000, -1)
print(spectral_data_flattened.shape) # no need to do this tbh since it's already flat

torch.Size([10000, 36])
tensor([0., 0., 0., 5., 0., 0., 2., 0., 0., 0., 5., 0., 1., 0., 3., 2., 5., 5.,
        2., 2., 1., 3., 0., 6., 0., 0., 3., 3., 1., 0., 0., 2., 0., 0., 5., 0.])
torch.Size([10000, 20000])


In [81]:
from sklearn.model_selection import train_test_split

spec_train, spec_test, matrix_train, matrix_test = train_test_split(spectral_data_flattened, Interaction_matrices_flattened, test_size=0.2, random_state=42)
spec_test, spec_val, matrix_test, matrix_val = train_test_split(spec_test, matrix_test, test_size=0.5, random_state=42)

print(spec_train.shape)
print(spec_test.shape)
print(spec_val.shape)

torch.Size([8000, 20000])
torch.Size([1000, 20000])
torch.Size([1000, 20000])


In [82]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F


# pytorch dataset and loaders
train_dataset = TensorDataset(torch.Tensor(spec_train), torch.Tensor(matrix_train))
val_dataset = TensorDataset(torch.Tensor(spec_val), torch.Tensor(matrix_val))
test_dataset = TensorDataset(torch.Tensor(spec_test), torch.Tensor(matrix_test))

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# defining the SVM

class SVMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SVMModel, self).__init__()
        self.hidden1= nn.Linear(input_dim, hidden_dim)
        self.act1 = nn.ReLU()
        self.hidden2 = nn.Linear(hidden_dim, hidden_dim)
        self.act2 = nn.ReLU() # added a bunch of hidden layers
        self.hidden3 = nn.Linear(hidden_dim, hidden_dim)
        self.act3 = nn.ReLU() # activation functions
        self.output = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = F.relu(self.hidden1(x))
        x = F.relu(self.hidden2(x))
        x = F.relu(self.hidden3(x))
        x = self.output(x)
        return x





In [83]:
print(spec_train.shape)

torch.Size([8000, 20000])


In [85]:
input_dim = spectral_data_flattened.shape[1]
hidden_dim = 128 # hyperparameter
output_dim = Interaction_matrices_flattened.shape[1]
model = SVMModel(input_dim, hidden_dim, output_dim)

# defining the loss func tion and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001) # hyperparameter

train_loss = 0
num_epochs = 25 # hyperparameter
for epoch in range(num_epochs):
    for spec_train, matrix_train in train_loader:
        optimizer.zero_grad()
        outputs = model(spec_train)
        loss = criterion(outputs, matrix_train)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()


    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')
    #print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss}')




Epoch 1/25, Loss: 7.077150344848633
Epoch 2/25, Loss: 7.041650772094727
Epoch 3/25, Loss: 6.055182933807373
Epoch 4/25, Loss: 4.757182598114014
Epoch 5/25, Loss: 4.406103134155273
Epoch 6/25, Loss: 4.44008207321167
Epoch 7/25, Loss: 4.2076568603515625
Epoch 8/25, Loss: 4.439328193664551
Epoch 9/25, Loss: 4.605320930480957
Epoch 10/25, Loss: 4.464384078979492
Epoch 11/25, Loss: 4.423389434814453
Epoch 12/25, Loss: 4.4784345626831055
Epoch 13/25, Loss: 4.564752101898193
Epoch 14/25, Loss: 4.405340671539307
Epoch 15/25, Loss: 4.49427604675293
Epoch 16/25, Loss: 4.460813522338867
Epoch 17/25, Loss: 4.3172383308410645
Epoch 18/25, Loss: 4.444540977478027
Epoch 19/25, Loss: 4.444989204406738
Epoch 20/25, Loss: 4.3337883949279785
Epoch 21/25, Loss: 4.25850772857666
Epoch 22/25, Loss: 4.342062950134277
Epoch 23/25, Loss: 4.431821346282959
Epoch 24/25, Loss: 4.44091272354126
Epoch 25/25, Loss: 4.3321309089660645
